In [1]:
from tensorflow.keras.models import load_model
model = load_model('fake_news_lstm_model.h5')

In [2]:
import pickle
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = 200  # Replace with the value used during training
# Example new texts to classify
new_texts = ["""A group of a dozen politically engaged pastors gathered for a private dinner on Friday night to hear a unique story in presidential politics: Louisiana Governor Bobby Jindal’s journey from Hinduism to Protestant Christianity and, ultimately, his embrace of what he calls “evangelical Catholicism.” Over two hours, the 42-year-old governor shared his spiritual transformation. He recounted speaking with a high school classmate who sought to “save his soul,” secretly reading the Bible in his closet to avoid his parents’ scrutiny, and feeling deeply moved by a movie depicting Jesus on the cross during his senior year.

“I was struck, and struck hard,” Jindal told the pastors. “This was the Son of God, and He had died for our sins.” The dinner was part of Jindal’s strategic outreach to pastors in key presidential battleground states like Iowa and South Carolina. His goal was to establish a political base among religious leaders that could elevate his standing among Republican contenders for the 2016 presidential election. Despite being known in GOP circles for his expertise on policy matters such as health care, Jindal, a Rhodes Scholar and Brown University graduate, lacked an established network of activist supporters outside Louisiana.

To address this, Jindal has leaned into his religious experiences, appealing to the GOP’s influential bloc of religious conservatives, many of whom have yet to settle on a favorite candidate for the presidential race. While other potential contenders, including Senators Rand Paul (Ky.) and Ted Cruz (Tex.) and Indiana Governor Mike Pence, are courting the evangelical base, Jindal seemed to make significant inroads over the weekend during his visit to Lynchburg, Virginia. Known as a hub for evangelicals and home to Liberty University, founded by the late Rev. Jerry Falwell, Lynchburg provided a platform for Jindal to connect with this core constituency.

In addition to the private dinner, Jindal delivered a rousing commencement address to an audience of 40,000 Christian conservatives at Liberty University. In his speech, he emphasized his faith and criticized President Obama’s perceived attacks on religious freedom. The pastors who attended the dinner said Jindal’s personal stories left a lasting impression. “He has the convictions, and he knows how to communicate them,” said Brad Sherman of Solid Rock Christian Church in Coralville, Iowa, who had previously supported Mike Huckabee’s 2008 campaign. Another pastor, C. Mitchell Brooks of Second Baptist Church in Belton, South Carolina, said Jindal’s faith journey and values placed him “on a par” with Huckabee, who is both a Baptist preacher and a politician.

The pastors were invited to Lynchburg by the American Renewal Project, a nonprofit group that mobilizes evangelical Christians through voter guides, grassroots training, and get-out-the-vote initiatives. The organization’s founder, David Lane, has established a network of pastors in politically pivotal states like Iowa, Missouri, Ohio, and South Carolina. Lane has also arranged trips to Israel for politicians like Rand Paul to strengthen ties with evangelical leaders. This group of pastors, including Donald Wildmon, founder of the American Family Association, had limited prior interaction with Jindal. However, their interest in him grew after he publicly supported Phil Robertson of “Duck Dynasty” fame during a controversy over remarks Robertson made about homosexuality.

During his visit to Lynchburg, Jindal positioned himself as a staunch advocate for cultural and religious values. In his commencement address, he expressed support for Jason and David Benham, twin brothers and Liberty graduates whose HGTV show, “Flip It Forward,” was canceled following revelations about their opposition to same-sex marriage. Jindal called the cancellation an example of “intolerance from the entertainment industry.” He added, “If these guys had protested at the Republican Party convention, instead of canceling their show, HGTV would probably have given them a raise.”

Jindal also criticized the Obama administration over the legal challenges faced by Hobby Lobby, a Christian-owned craft store chain that opposed providing insurance coverage for certain contraceptives under the Affordable Care Act. Highlighting the family’s commitment to Christian values, Jindal noted their generosity as employers, including paying above minimum wage and raising salaries despite the economic recession. “None of this matters to the Obama administration,” Jindal told the audience.

For many pastors, Jindal’s personal story was the highlight of the weekend. Born Piyush Jindal in 1971, four months after his parents emigrated from India to Baton Rouge, Louisiana, Jindal adopted the name “Bobby” as a child, inspired by a character from the TV show The Brady Bunch. He explained to the pastors that his conversion to Christianity wasn’t a single moment of revelation but a gradual process. He recounted receiving a Bible as a Christmas gift from a childhood friend and initially dismissing it as an odd choice since “everyone knows you can get one for free in any hotel.” The gold lettering on the cover, however, made him reconsider.

His faith deepened during high school, where a conversation with a girl he hoped to date led him to explore her beliefs. He began reading the Bible in secret to avoid upsetting his parents. Jindal described a transformative moment when, while watching a film about Christ’s crucifixion, the Bible and his existence suddenly seemed clearer.

Although he briefly touched on his conversion to Catholicism during college, Jindal primarily focused on how his values align with those of other Christian conservatives. “I read the words of Jesus Christ, and I realized they were true,” Jindal told the graduates at Liberty. “I used to think I found God, but now I believe He found me."""]

# Convert texts to sequences using the tokenizer
new_sequences = tokenizer.texts_to_sequences(new_texts)

# Pad sequences to the same length as used during training
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Make predictions using the trained model
predictions = model.predict(new_padded_sequences)

# Print predictions
# Convert probabilities to binary predictions
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# Print binary predictions
print(binary_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
[[0]]


In [4]:
from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

# Load the model and tokenizer
model = tf.keras.models.load_model('fake_news_lstm_model.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Recompile to suppress warning

with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define the maximum sequence length (same as during training)
max_sequence_length = 200

# Initialize Flask app
app = Flask(__name__)

# Define the home route to serve the frontend
@app.route('/')
def home():
    return render_template('index.html')

# Define the prediction route
@app.route('/predict', methods=['POST'])
def predict():
    # Get the input text from the request
    data = request.json
    text = data.get('text', '')

    # Preprocess the input text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post', truncating='post')

    # Make a prediction
    prediction = model.predict(padded_sequence)
    probability = float(prediction[0][0])  # Convert numpy float to Python float

    # Return the prediction as JSON
    return jsonify({
        'text': text,
        'probability': probability,
        'prediction': 'Fake News' if probability > 0.5 else 'Real News'
    })

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=False)  # Disable debug mode for production

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 15:10:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 15:10:55] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step


INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 15:11:06] "POST /predict HTTP/1.1" 200 -
